Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [19]:
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

In [20]:
df_clean = pd.read_csv("dataset/clean_data.csv")

# df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [21]:
X = df_clean[['input', 'average_runtime', 'speedup', 'efficiency']].values
y = df_clean['cores'].values

### Regression Models

#### MLP Regression Model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [23]:
learning_rate = 0.00001
input_shape = (X_train.shape[1],)

callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=1000, verbose=2, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/1000
7/7 - 0s - loss: 345.6714 - val_loss: 352.2441 - 377ms/epoch - 54ms/step
Epoch 2/1000
7/7 - 0s - loss: 341.1778 - val_loss: 347.5164 - 22ms/epoch - 3ms/step
Epoch 3/1000
7/7 - 0s - loss: 336.7672 - val_loss: 343.4056 - 22ms/epoch - 3ms/step
Epoch 4/1000
7/7 - 0s - loss: 333.2195 - val_loss: 339.2698 - 22ms/epoch - 3ms/step
Epoch 5/1000
7/7 - 0s - loss: 330.0306 - val_loss: 335.1895 - 23ms/epoch - 3ms/step
Epoch 6/1000
7/7 - 0s - loss: 326.1782 - val_loss: 331.5050 - 22ms/epoch - 3ms/step
Epoch 7/1000
7/7 - 0s - loss: 322.8717 - val_loss: 327.8490 - 22ms/epoch - 3ms/step
Epoch 8/1000
7/7 - 0s - loss: 319.5274 - val_loss: 324.3533 - 23ms/epoch - 3ms/step
Epoch 9/1000
7/7 - 0s - loss: 316.3391 - val_loss: 320.7977 - 30ms/epoch - 4ms/step
Epoch 10/1000
7/7 - 0s - loss: 313.1501 - val_loss: 316.6976 - 26ms/epoch - 4ms/step
Epoch 11/1000
7/7 - 0s - loss: 309.0244 - val_loss: 311.8205 - 22ms/epoch - 3ms/step
Epoch 12/1000
7/7 - 0s - loss: 304.6334 - val_loss: 306.5490 - 22ms/epoc

In [24]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

3/3 [==============================] - 0s 1ms/step - loss: 2.2960
Erro no conjunto de teste: 2.295973062515259
3/3 [==============================] - 0s 998us/step
Valor Real: 6, Previsão: 6.810382843017578
Valor Real: 29, Previsão: 29.833805084228516
Valor Real: 19, Previsão: 21.315561294555664
Valor Real: 23, Previsão: 22.88559913635254
Valor Real: 27, Previsão: 27.60814666748047
Valor Real: 6, Previsão: 4.6508588790893555
Valor Real: 2, Previsão: 2.4595413208007812
Valor Real: 1, Previsão: -0.017200589179992676
Valor Real: 18, Previsão: 18.294673919677734
Valor Real: 31, Previsão: 30.93781280517578


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [25]:
loo = LeaveOneOut()
mse_scores = []
callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=100, verbose=0, callbacks=[callback])

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

print(f'Erro médio quadrático usando LOOCV: {min(mse_scores)}')

1/1 [==============================] - 0s 30ms/step
Erro médio quadrático usando LOOCV: 2.661918188096024e-07


In [26]:
x_test_new = [
    [1,0.24561333656311035,1.0264687524571021,0.03207714851428444],
    [1,0.23585212230682373,1.0689512254665345,0.042758049018661384]
]

model.predict(x=x_test_new)

1/1 [==============================] - 0s 26ms/step


array([[65.08774 ],
       [63.754257]], dtype=float32)